# Importing ORACC Data from corpus.json
by Niek Veldhuis
UC Berkeley

February-May 2017

# NB work in progress - new parser using requests.json()
this replaces json-corpus.ipynb
# All introductory text to be replaced etc.
# Remove redundant cells

# TODO
* check that COFs are treated properly
* check that lines that continue into the next line (as in bilinguals) are captured completely. Such lines are indicated in the json by the the addition of 'l' (lower case L) to the reference (.ref).



# Introduction

Purpose of the code is to download [ORACC](http://oracc.org) JSON files that contain textual data and to produce a `.csv` file with the relevant data for use in computational text analysis. This comes in the place of scraping the published `html` (see the [Scrape-ORACC](https://github.com/niekveldhuis/Digital-Assyriology/tree/master/Scrape-Oracc) repo). The JSON files contain all the transliteration and lemmatization data of an ORACC project as well as metadata . For an introduction to the various ORACC JSON files see the [ORACC Open Data](http://oracc.org/doc/opendata) page.

The resulting data file may include various elements of the ORACC data structure. The current code will output a file with the following fields: 

* id_line
* label
* lemma
* base
* extent
* scope

The fields `extent` and `scope` capture the number of missing lines or columns.

The selection of fields may be adjusted with standard `Pandas` functions.

## Notes
The current version of the script works with the `ijson` library. Documentation for [ijson](https://www.dataquest.io/blog/python-json-tutorial/), unfortunately, is extremely brief. It is likely that in the near future, because of considerations of space, [ORACC](http://oracc.org) will no longer make available individual `.json` files, but only the file `json.zip`, a compressed file that includes all the `.json` files that belong to a single project. For that reason the current code will first download the `json.zip` and then extract the relevant files (at the moment of writing this note it is still possible to directly download such files from the [ORACC](http://oracc.org) server. 

This notebook is written for **Python 3.5** with **Pandas 0.19** and **ijson 2.3**.

The first section of this notebook will download and parse data from any [ORACC](http://oracc.org) project (or combination of projects). For most purposes, the number of data elements extracted will be too large and it will be necessary to select and manipulate the data set. The second section of the notebook selects only proper nouns (personal names, royal names, geographical names, etc.) and prepares the data for usage in Social Network Analysis software. This is only one example of how the data may be used, other usages could include topic modeling, word2vec, etc. Each of those analyses requires a specific data format and therefore specific data manipulation. Further examples of how such manipulation might work are foreseen for a later version of this notebook.

The initial version of this notebook was written for the [Phylogenetics](https://github.com/ErinBecker/digital-humanities-phylogenetics) project with Erin Becker of [Data Carpentry](http://www.datacarpentry.org). 

## Licensing
This notebook may be downloaded, used and adapted without any restrictions.

In [3]:
import pandas as pd   
import requests
import zipfile
import tqdm
import numpy as np

# Input List of Text IDs or a project abbreviation
Identify a list of text IDs (P, Q, and X numbers) in the directory `input`. The IDs are six-digit P, Q, or X numbers preceded by a project abbreviation in the format 'PROJECT/P######' or 'PROJECT/SUBPROJECT/Q######'. For example:
* dcclt/P117395
* etcsri/Q001203
* rinap/rinap1/Q003421

The list should be created with a flat text editor such as Textedit or Emacs, and the filename should end in `.txt`.

Alternatively, one may enter the name (abbreviation) of a project or sub-project in [ORACC](http://oracc.org) and pull all the lemmatized data from that project. Note that the script will not automatically pull data from subprojects, they have to be requested separately. Examples:
* saao/saa01
* aemw/amarna
* rimanum

In [2]:
name = input('Filename or project abbreviation: ')

Filename or project abbreviation: obmc


In [19]:
if name[-4:] == '.txt':
    textids = 'text_ids/' + name
    with open(textids, 'r') as f:
        pqxnos = f.readlines()
    pqxnos = [x.strip() for x in pqxnos]  # strip spaces left and right
    pqxnos = [x for x in pqxnos if not x == ""] # strip empty lines
#    pqxnos = [x[-7:].upper() for x in pqxnos]
    projects = [x[:-8].lower() for x in pqxnos]
    projects = list(set(projects))
else:
    name = name.strip().lower()
    projects = [name]
    url = "http://oracc.org/" + name + "/corpus.json"
    r = requests.get(url)
    corpus = r.json()
    pqxnos = list(corpus["members"].keys())
    pqxnos = [name + '/' + no for no in pqxnos]

In [20]:
projects

['obmc']

## 1.2 Create Download Directory and JSON directory
For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist)

In [21]:
import errno
import os
try:
    os.mkdir('jsonzip')
except OSError as exc:
    if exc.errno !=errno.EEXIST:
        raise
    pass
try:
    os.mkdir('json')
except OSError as exc:
    if exc.errno !=errno.EEXIST:
        raise
    pass

## 1.3 Download `json.zip`
For each project from which files are to be processed download the entire project (all the json files) in `https://github.com/oracc/json`. The file is called `PROJECT.zip` (for instance: `dcclt.zip`). For subprojects the file is called `PROJECT-SUBPROJECT.zip` (for instance `cams-gkab.zip`). 

For larger projects (such as [DCCLT](http://oracc.org/dcclt)) the `zip` file may be 25Mb or more. Downloading may take some time and it may be necessary to chunk the downloading process. The `iter_content()` function in the `requests` library takes care of that.

Although downloading the entire zip file is time consuming, it will make processing the individual files much more efficient and the code is less likely to break due to interruption in connectivity.

In [22]:
CHUNK = 16 * 1024
for project in tqdm.tqdm(projects):
    project = project.replace('/', '-')
    url = "https://raw.github.com/oracc/json/master/" + project + ".zip"
    file = 'jsonzip/' + project + '.zip'
    print("Downloading " + url + " saving as " + file)
    r = requests.get(url)
    with open(file, 'wb') as f:
        for c in r.iter_content(chunk_size=CHUNK):
            f.write(c)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  1.19it/s]


## 1.4 Extract JSON files from `json.zip`
Extract the texts listed in the list of text IDs from the `json.zip`. All files are extracted to a directory called `data/[PROJECT]/json/corpusjson` (for instance `data/dcclt/json/corpusjson`). If the file belongs to a subproject the directory is called `data/[PROJECT]/[SUBPROJECT]/json/corpusjson`. 

In [23]:
target_dir = 'json'
for no in tqdm.tqdm(pqxnos):
    project = no[:-8].lower()
    pno = no[-7:].upper()
    zip_file = "jsonzip/" + project.replace('/', '-') + ".zip"
    with zipfile.ZipFile(zip_file,"r") as zip_ref:
        file = project + '/corpusjson/' + pno + '.json'
        try:
            zip_ref.extract(file, target_dir)
        except:
            print(no + ' is not available')

100%|██████████| 91/91 [00:00<00:00, 230.56it/s]


## 1.5 Parse JSON files
# This should be replaced by some version of parsejson() in First_JSON_parser
The `parsejson()` below includes "id_word" which has the form `TextID.LineID.WordID` - in other words, line and text ID can be derived from it.

For the caller function make sure that texts from different projects will be parsed correctly (does pqxnos include a project?).

========================

The function `oraccjasonparser()` takes one argument (the **ID** or **P/Q/X-number** of the `.json` file). It looks for the prefix `textid` to retrieve the six-digit P, Q, or X number of the text artifact. Parsing the file sequentially the code looks for the places where a line starts (`'.type' = 'line-start'`) and where a word starts (`'.node' = 'l'`, where `l` is for "lemma"). At each level the code will retrieve the relevant data and create a list where each entry is a dictionary that represents a single word. 

Words not only include lemmatized words, but also unlemmatized and unlemmatizable words (such as breaks).

The dictionary includes the keys `id_line` and `id_word` that allow the user to reassemble words and lines in order.

In [67]:
def parsejson(text, lemm_l = None):
    if lemm_l == None:
        lemm_l = []
    for dictionary in text["cdl"]:
        if "cdl" in dictionary: 
            parsejson(dictionary, lemm_l)
        if "f" in dictionary:
            lemma = dictionary["f"]
            lemma["id_word"] = dictionary["ref"]
            lemm_l.append(lemma)
        if "strict" in dictionary and dictionary["strict"] == "1":
            keys = ["extent", "scope","state"]
            lemma = {key: dictionary[key] for key in keys}
            lemma["id_word"] = dictionary["ref"] + ".0"
            lemm_l.append(lemma)
    return lemm_l

In [68]:
url = "http://oracc.org/dcclt/corpusjson/P273880.json"
r = requests.get(url).json()
lemm = parsejson(r)

In [69]:
df = pd.DataFrame(lemm)
df

,base,cf,cont,delim,epos,extent,form,gdl,gw,id_word,lang,morph,norm0,pos,scope,sense,state
0,NaN,NaN,NaN,NaN,NaN,beginning,NaN,NaN,NaN,P273880.4.0,NaN,NaN,NaN,NaN,column,NaN,missing
1,NaN,NaN,NaN,NaN,NaN,NaN,{ŋeš}x-x,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",NaN,P273880.5.1,sux,NaN,NaN,u,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,{ŋeš}x-x,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",NaN,P273880.6.1,sux,NaN,NaN,u,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,{ŋeš}x-x,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",NaN,P273880.7.1,sux,NaN,NaN,u,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,{ŋeš}x-x-x,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",NaN,P273880.8.1,sux,NaN,NaN,u,NaN,NaN,NaN
5,{ŋeš}al-la-nu-um,allanum,NaN,NaN,N,NaN,{ŋeš}al-la-nu-um,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",oak,P273880.9.1,sux,~,allanum,N,NaN,hazelnut,NaN
6,{ŋeš}ha-lu-ub₂,halub,NaN,NaN,N,NaN,{ŋeš}ha-lu-ub₂,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",tree,P273880.10.1,sux,~,halub,N,NaN,a kind of cherry(tree),NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,{ŋeš}x-TUG₂,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",NaN,P273880.11.1,sux,NaN,NaN,u,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,{ŋeš}x-x,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",NaN,P273880.12.1,sux,NaN,NaN,u,NaN,NaN,NaN
9,{ŋeš}šim-gig,šimgig,NaN,NaN,N,NaN,{ŋeš}šim-gig,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",tree,P273880.13.1,sux,~,šimgig,N,NaN,an aromatic tree,NaN


In [ ]:
def oraccjsonparser(text_id):
    project = text_id[:-8]
    PQXno = text_id[-8:]
    filename = 'json/' + project + '/corpusjson/' + PQXno +'.json'
    
    lemma_fields = ["base", "cf", "cont", "epos", "frag", "form", "gw", "inst", "lang", "morph", "norm", 
              "norm0", "pos", "sense", "sig"] # these are the fields that constitute a lemma
                                            # it does not include the gdl fields that define a sign
    dollar_fields = ["extent", "scope", "state"] # relevant fields in $-lines 
    
    with open(filename, 'r') as d:
        parser = ijson.parse(d)
        
        word_l = []
        word_d = {}
        line_start = False
        word_start = False
        nonx = False
        
        for prefix, event, value in parser:
            field = prefix[prefix.rfind('.')+1:] # get element after the last dot of the prefix

            if (event, value) == ("map_key", "node"):
                word_start = False # at the beginning of a new node
                line_start = False # reset all flags
                nonx = False
                if not word_d == {}: # and save the preceding word - if there is anything in it
                    word_l.append(word_d)
                    word_d = {}  # then start a new dictionary for a new word
                    
            if value == 'line-start':
                line_start = True
            if field == 'node' and value == 'l':
                word_start = True
            if value == "nonx":
                nonx = True

            if prefix == 'textid':
                id_text = value

            if line_start:
                if field == 'ref':
                    id_line = value # id_line is a reference number for a line
                                    # that includes the id_text (e.g. P123456.49)
                if field == 'label':
                    label = value   # label is a human-readable line number of the format
                                    # o ii 24' (obverse column 2 line 24')

            if word_start:
                word_d['id_text'] = id_text # provide each word with appropriate 
                word_d['id_line'] = id_line # text and line-ID
                word_d['label'] = label     # and the line label.

                if field == 'ref':
                    word_d['id_word'] = value
                if not "gdl" in prefix:
                    if field in lemma_fields:  # lemma-fields is a list of relevant field names
                        word_d[field] = value # copy each element into the dictionary
                    
            if field == 'gdl_utf8':
                if 'cuneify' in word_d:           # if the key 'gdl_utf8' already exists for this word
                    word_d['cuneify'] = word_d['cuneify'] + value  # add the new value to the existing one
                else:                              # else create the key and add the value
                    word_d['cuneify'] = value
                    
            if nonx:                # this captures so-called $-lines with information
                if field == 'ref':  # about number of broken lines/columns.
                    id_line = value          # $-lines have their own id_line.
                if field == 'strict':
                    if value == '1': # select only 'strict' $ lines
                        word_d['id_line'] = id_line
                        word_d['id_text'] = id_text
                    else:
                        nonx = False
                if field in dollar_fields: # capture the three elements of strict $ lines
                    word_d[field] = value   # namely extent, scope, and state.
    if not word_d == {}:
        word_l.append(word_d)  # make sure that the last word is captured, too.
    return(word_l) # return a list of dictionaries, where each entry (dictionary) in
                   # the list represents a word.

## 1.6 Call the Parser Function for Each Textid

In [ ]:
word_l = []
for id_text in tqdm.tqdm(pqxnos):
    try:
        word_l.extend(oraccjsonparser(id_text))
    except:
        print(no + ' is not available or not complete')

## 1.7 Transform the Data into a DataFrame
The word_l list is transformed into a Pandas dataframe for further manipulation.

For various reasons not all JSON files will have all data types that potentially exist in an [ORACC](http://oracc.org) signature. Only Sumerian words have a `base`, so if your data set has no Sumerian, this column will not exist in the DataFrame.  If a text has no breakage information in the form of `$ 1 line broken` (etc.) the fields `extent`, `scope`, and `state` do not exist. Since such fields are referenced in the code below (sections 2-4) the next cell will check for the existence of each column and create an empty column if necessary.

In [ ]:
words = pd.DataFrame(word_l)
fields = ['base', 'cf', 'cont', 'epos', 'extent', 'form', 'gw', 'id_line', 'id_text', 'id_word',
          'label', 'lang', 'morph', 'norm', 'norm0', 'pos', 'scope', 'sense', 'sig']
for field in fields:
    if not field in words.columns:
        words[field] = ''
words = words.fillna('') # replace Missing Values by empty string
words.head(100)

## 1.8 Remove Spaces and Commas from Guide Word and Sense
Spaces in Guide Word and Sense may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens or nothing, respectively.

In [ ]:
words['sense'] = [x.replace(' ', '-') for x in words['sense']]
words['sense'] = [x.replace(',', '') for x in words['sense']]
words['gw'] = [x.replace(' ', '-') for x in words['gw']]
words['gw'] = [x.replace(',', '') for x in words['gw']]

The columns in the resulting DataFrame correspond to the elements of a full [ORACC](http://oracc.org) signature, plus information about text, line, and word ids:
* base (Sumerian only)
* cf (Citation Form)
* cont (continuation of the base; Sumerian only)
* epos (Effective Part of Speech)
* form (transliteration, omitting all flags such as indication of breakage)
* frag (transliteration; including flags)
* gdl_utf8 (cuneiform)
* gw (Guide Word: main or first translation in standard dictionary)
* id_line (a line ID that begins with the six-digit P, Q, or X number of the text)
* id_text (six-digit P, Q, or X number)
* id_word (word ID that begins with the ID number of the line)
* label (traditional line number in the form o ii 2' (obverse column 2 line 2'), etc.)
* lang (language code, including sux, sux-x-emegir, sux-x-emesal, akk, akk-x-stdbab, etc)
* morph (Morphology; Sumerian only)
* norm (Normalization: Akkadian)
* norm0 (Normalization: Sumerian)
* pos (Part of Speech)
* sense (contextual meaning)
* sig (full ORACC signature)

Not all data elements (columns) are available for all words. Sumerian words never have a `norm`, Akkadian words do not have `norm0`, `base`, `cont`, or `morph`. Most data elements are only present when the word is lemmatized; only `lang`, `form`, `pos`, `id_word`, `id_line`, and `id_text` should always be there. An unlemmatized word has `pos` 'X' (for unknown). Broken words have `pos` 'u' (for 'unlemmatizable).

# 2. Manipulate for SNA
The columns of the `words` DataFrame may be manipulated with standard Pandas methods to create the desired output. By way of example, the following code will select proper nouns only and create two `.csv` files (`edges.csv` and `nodes.csv`) that may be ingested by Social Network Analysis (SNA) software. The column names follow the conventions used in [Gephi](https://gephi.org/).

## 2.1 Select Proper Nouns
First list all Part of Speech tags currently available in the corpus.

In [ ]:
pos = list(set(words['pos']))
pos

Then list the tags that are relevant in the list `pos` and use that list to select the rows of the DataFrame that contain proper nouns.  

In [ ]:
pos = ['CN', 'DN', 'EN', 'FN', 'GN', 'PN', 'NN', 'RN', 'SN', 'TN', 'WN'] # what is 'NN'?
proper_nouns = words.loc[words['pos'].isin(pos)].reset_index(drop=True)
proper_nouns.head()

## 2.2 Keep  Norm, Pos, and id_text
Now select the relevant columns.

In [ ]:
proper_nouns = proper_nouns[['norm', 'pos', 'id_text']].drop_duplicates()
proper_nouns = proper_nouns[proper_nouns['norm'] != ''].reset_index(drop=True)
proper_nouns.head()

## 2.3 Create Edge List
The edge list contains the columns `source` and `target` and combines all proper nouns that appear in a single text as source-target pairs. All edges are considered `undirected`.

In [ ]:
edges = []
for i in tqdm.tqdm(range(len(proper_nouns))):
    for j in range(i+1, len(proper_nouns)):
        if proper_nouns['id_text'][i] == proper_nouns['id_text'][j]:
            edge = [proper_nouns['norm'][i], proper_nouns['norm'][j]]
            edges.append(edge)
        else:
            break
edges_df = pd.DataFrame(edges)
edges_df.columns = ['source', 'target']
edges_df['type'] = 'undirected'
edges_df.head()

In [ ]:
with open("output/edges.csv", 'w') as f:
    edges_df.to_csv(f, index=False)

## 2.4 Create Node List
pn_set contains the unique proper nouns in the entire corpus. This become the node list in Gephi

In [ ]:
pn_set = proper_nouns[['norm', 'pos']].drop_duplicates() # Assur DN and Assur GN are not considered duplicates!
pn_set.columns = ['Id', 'Type']
pn_set['Label'] = pn_set['Id']
pn_set.head()

In [ ]:
with open("output/nodes.csv", 'w') as f:
    pn_set.to_csv(f, index=False)

# 3. Manipuate for Analysis on Line level (e.g. phylogenetics)
For analyses that use a line as unit of analysis (e.g. lines in lexical texts as analyzed in the [Phylogenetics](https://github.com/ErinBecker/digital-humanities-phylogenetics) project) one may need to create lemmas and combine these into lines by using the `id_line` variable.

## 3.1 Create Lemmas and Adjust Bases
A lemma, [ORACC](http://oracc.org) style, combines Citation Form, GuideWord and POS into a unique reference to one particular lemma in a standard dictionary, as in `lugal[king]N` (Sumerian) or `šarru[king]N`. Usually, not all words in a text are lemmatized, because a word may be (partly) broken and/or unknown. Unlemmatized and unlemmatizable words will receive a place-holder lemmatization that consists of the transliteration of the word (instead of the Citation Form), with `NA` as GuideWord and POS, as in `i-bu-x[NA]NA`. Note that `NA` is a string.

For Sumerian projects each lemmatized word has a `base` (the word without morphology). For non-lemmatized words a place-holder base is created that consists of the transliteration of the word. If you are not working with Sumerian data.

In [ ]:
words["lemma"] = words.apply(lambda r: (r["cf"] + '[' + r["gw"] + ']' + r["pos"]) 
                            if r["cf"] != '' else r['form'] + '[NA]NA', axis=1)
words['lemma'] = [lemma if not lemma == '[NA]NA' else '' for lemma in words['lemma'] ]
words['base'] = words.apply(lambda r: r["base"] if r["base"] != '' or r['label'] == '' else r['form'], axis=1)

## 3.2 Group by Line
In the `words` dataframe each word has a separate row. In order into change this to a line-by-line representation we use the Pandas `.groupby` function, using the `id_line` and `label` fields as arguments (`id_line` has an abstract number that indicates the sequence of lines in a text object; `label` is a human-readable line number in the format `o ii 3`: obverse column 2, line 3). The fields that are aggregated are `lemma`, `base`, `extent`, and `scope`. The fields `extent` and `scope` represent data on the number of broken lines. If you work with Akkadian data you want to leave out the field `base`.

In [ ]:
lines = words.groupby([words['id_line'], words['label']]).agg({
        'lemma': ' '.join,
        'base': ' '.join,
        'extent': ''.join, 
        'scope': ''.join
    }).reset_index()
lines        

Note that `id_line` is a string variable and therefore does not give the lines in the right order. We should split `id_line` into two variables: `id_text` (the first 7 characters; we lost the old `id_text` column in the `.groupby` function above) and a new `line` variable, which is a number. 

In [ ]:
lines['id_text'] = lines['id_line'].str[:7] # id_text was lost in the grouping above and is recreated
lines['line'] = [line[line.rfind('.')+1:] for line in lines['id_line']] #create a line number for sorting
lines['line'] = [x.replace('l', '') for x in lines['line']]
lines['line'] = [int(x) if not x == '' else np.nan for x in lines['line']]
lines = lines.sort_values(['id_text', 'line']).reset_index(drop=True)
lines.head(100)

Note that the new `line` field is not a line number in the traditional sense of the word (this is `label`) but a number used to organize lines in the appropriate order.

## 3.3 Save in CSV Format

In [ ]:
filename = filename[:-4]
with open('output/' + filename + '.csv', 'w') as w:
    lines.to_csv(w, encoding='utf8')

# 4 Manipulate for Document-level Analysis
For analyses that use documents in a Document Term Matrix or otherwise a similar type of manipulation is needed. This output may be used in Word2vec, in Topic Modeling and in other types of algorithms. First lemmas and bases are dealt with in the same way as above, section 3.1.

## 4.1 Create Lemmas and Adjust Bases
A lemma, [ORACC](http://oracc.org) style, combines Citation Form, GuideWord and POS into a unique reference to one particular lemma in a standard dictionary, as in `lugal[king]N` (Suerian) or `šarru[king]N`. Usually, not all words in a text are lemmatized, because a word may be (partly) broken and/or unknown. Unlemmatized and unlemmatizable words will receive a place-holder lemmatization that consists of the transliteration of the word (instead of the Citation Form), with `NA` as GuideWord and POS, as in `i-bu-x[NA]NA`. Note that `NA` is a string.

For Sumerian projects each lemmatized word has a `base` (the word without morphology). For non-lemmatized words a place-holder base is created that consists of the transliteration of the word. If you are not working with Sumerian data.

In [ ]:
words['lemma'] = words['cf'] # first element of lemma is the citation form
words['lemma'] = [words['lemma'][i] + '[' + words['gw'][i] 
                     + ']' + words['pos'][i] 
                     if not words['lemma'][i] == '' 
                     else words['form'][i] +'[NA]NA' for i in range(len(words))]
words['lemma'] = [lemma if not lemma == '[NA]NA' else '' for lemma in words['lemma'] ]
words['base'] = [words['base'][i] if not words['base'][i] == '' 
                 or words['label'][i] == '' else words['form'][i] 
                 for i in range(len(words))]

# 4.1 Group by Document
In order to group by Document we use the field `id_text` and aggregate `lemma` and `base`.

In [ ]:
documents = words.groupby(words['id_text']).agg({
        'lemma': ' '.join,
        'base': ' '.join,
    }).reset_index()
documents

## 3.4 Save in CSV Format

In [ ]:
filename = filename[:-4]
with open('output/' + filename + '.csv', 'w') as w:
    lines.to_csv(w, encoding='utf8')

## 3.5 Tokenizing
Since lemmas do not contain spaces (see above, section 1.8) tokenizing is extremely easy and basically consists of splitting on spaces. Tokenized data are used in Topic Modeling, Word2vec, etc. Tokenizing is also necessary for making a Document Term Matrix.

In [ ]:
documents['tokens'] = documents['lemma'].str.split()
documents

# 4 Test Area

In [131]:
with open('text_ids/test.txt', 'r') as f:
    pqxnos = f.read().splitlines()

In [132]:
pqxnos = [no.strip() for no in pqxnos]

In [133]:
nos_labels = [no.split(' ', 1) if " " in no else [no, '-'] for no in pqxnos]

In [134]:
for label in nos_labels:
    label[1] = label[1].split('-')

In [135]:
nos_labels

[['dcclt/P225059', ['', '']],
 ['dcclt/P251686', ['o 1 ', ' o 8']],
 ['dcclt/P251867', ['', '']],
 ['dcclt/P296528', ['', " r v 10'"]],
 ['dcclt/P228683', ['r 1 ', '']]]

In [136]:
##this parsejson takes as second argument a list [startlabel, endlabel]
#
def parsejson(text, labels = ["", ""], lemm_l = None):
    label = ""
    dollar_keys = ["extent", "scope","state"]
    startlabel = labels[0].strip()
    endlabel = labels[1].strip()
    if startlabel == "":    
        keep = True
    else:
        keep = False
    if lemm_l == None:
        lemm_l = []
    for dictionary in text["cdl"]:
        if "cdl" in dictionary: 
            parsejson(dictionary, labels, lemm_l)
        if "label" in dictionary:
            label = dictionary["label"]
        if label == startlabel:
            keep = True
        if label == endlabel:
            keep = False
        if keep == True or label == endlabel: # the "or" statement ensures that the line
            if "f" in dictionary:             # corresponding to the endlabel is included.
                lemma = dictionary["f"]
                lemma["id_word"] = dictionary["ref"]
                lemm_l.append(lemma)
            if "strict" in dictionary and dictionary["strict"] == "1":
                lemma = {key: dictionary[key] for key in dollar_keys}
                lemma["id_word"] = dictionary["ref"] + ".0"
                lemm_l.append(lemma)
    return lemm_l

In [137]:
lemm = []
for pqx in nos_labels:
    project = pqx[0][:-8]
    textid = pqx[0][-7:]
    labels = pqx[1]
    url = "http://oracc.org/" + project + "/corpusjson/" + textid + ".json"
    print(url, labels)   
    r = requests.get(url).json()
    lemm.extend(parsejson(r, labels))

http://oracc.org/dcclt/corpusjson/P225059.json ['', '']
http://oracc.org/dcclt/corpusjson/P251686.json ['o 1 ', ' o 8']
http://oracc.org/dcclt/corpusjson/P251867.json ['', '']
http://oracc.org/dcclt/corpusjson/P296528.json ['', " r v 10'"]
http://oracc.org/dcclt/corpusjson/P228683.json ['r 1 ', '']


In [138]:
df = pd.DataFrame(lemm)
df

,base,cf,cont,delim,epos,extent,form,gdl,gw,id_word,lang,morph,norm0,pos,scope,sense,state
0,{ŋeš}emerah,emerah,NaN,NaN,N,NaN,{ŋeš}emerah,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",jug,P225059.4.1,sux,~,emerah,N,NaN,jug,NaN
1,{ŋeš}emerah,emerah,NaN,,N,NaN,{ŋeš}emerah,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",jug,P225059.5.1,sux,~,emerah,N,NaN,jug,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,x,"[{'x': 'ellipsis', 'id': 'P225059.5.2.0', 'bre...",NaN,P225059.5.2,sux,NaN,NaN,u,NaN,NaN,NaN
3,{ŋeš}emerah,emerah,NaN,,N,NaN,{ŋeš}emerah,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",jug,P225059.6.1,sux,~,emerah,N,NaN,jug,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,x,"[{'x': 'ellipsis', 'id': 'P225059.6.2.0', 'bre...",NaN,P225059.6.2,sux,NaN,NaN,u,NaN,NaN,NaN
5,{ŋeš}gu-za,guza,NaN,NaN,N,NaN,{ŋeš}gu-za,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",chair,P251686.4.1,sux,~,guza,N,NaN,chair,NaN
6,{ŋeš}gu-za,guza,NaN,,N,NaN,{ŋeš}gu-za,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",chair,P251686.5.1,sux,~,guza,N,NaN,chair,NaN
7,šu-niŋin₂,šuniŋin,NaN,NaN,N,NaN,šu-niŋin₂,"[{'v': 'šu', 'id': 'P251686.5.2.0', 'delim': '...",total,P251686.5.2,sux,~,šuniŋin,N,NaN,total,NaN
8,{ŋeš}gu-za,guza,NaN,,N,NaN,{ŋeš}gu-za,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",chair,P251686.6.1,sux,~,guza,N,NaN,chair,NaN
9,zag,zag,NaN,,N,NaN,zag-bi,"[{'v': 'zag', 'id': 'P251686.6.2.0', 'delim': ...",side,P251686.6.2,sux,"~,bi","zag,bi",N,NaN,side,NaN


In [81]:
import pandas as pd


In [119]:
url = "http://oracc.org/dcclt/corpusjson/P296528.json"
r = requests.get(url).json()
lemm = parsejson(r, ["", "r v 10'"])

In [123]:
df = pd.DataFrame(lemm)
len(df)

22

In [124]:
df

,base,cf,delim,epos,extent,form,gdl,gw,id_word,lang,morph,norm0,pos,scope,sense,state
0,NaN,NaN,NaN,NaN,NaN,{ŋeš}IGI-TUR,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",NaN,P296528.2.1,sux,NaN,NaN,X,NaN,NaN,NaN
1,{ŋeš}ligima,ligima,NaN,N,NaN,{ŋeš}ligima,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",shoot,P296528.3.1,sux,~,ligima,N,NaN,"shoot, sapling",NaN
2,{ŋeš}ŋešnimbar,ŋešnimbar,NaN,N,NaN,{ŋeš}ŋešnimbar,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",palm,P296528.4.1,sux,~,ŋešnimbar,N,NaN,date palm,NaN
3,{ŋeš}ŋešnimbar,ŋešnimbar,,N,NaN,{ŋeš}ŋešnimbar,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",palm,P296528.5.1,sux,~,ŋešnimbar,N,NaN,date palm,NaN
4,suhušₓ(TUR),suhuš,NaN,N,NaN,suhušₓ(TUR),"[{'q': 'suhušₓ(TUR)', 'id': 'P296528.5.2.0', '...",offshoot,P296528.5.2,sux,~,suhuš,N,NaN,date palm offshoot,NaN
5,{ŋeš}ŋešnimbar,ŋešnimbar,,N,NaN,{ŋeš}ŋešnimbar,"[{'det': 'semantic', 'pos': 'pre', 'seq': [{'v...",palm,P296528.6.1,sux,~,ŋešnimbar,N,NaN,date palm,NaN
6,tur,tur,NaN,V/i,NaN,tur,"[{'v': 'tur', 'id': 'P296528.6.2.0'}]",small,P296528.6.2,sux,~,tur,V/i,NaN,small,NaN
7,NaN,NaN,NaN,NaN,rest,NaN,NaN,NaN,P296528.7.0,NaN,NaN,NaN,NaN,column,NaN,missing
8,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,P296528.11.0,NaN,NaN,NaN,NaN,column,NaN,missing
9,NaN,NaN,NaN,NaN,beginning,NaN,NaN,NaN,P296528.13.0,NaN,NaN,NaN,NaN,column,NaN,missing
